In [1]:

! pip install utils
! pip install paramiko
! pip install nltk

     |████████████████████████████████| 215kB 5.3MB/s 
     |████████████████████████████████| 3.2MB 9.3MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 962kB 38.0MB/s 


In [2]:
! pip  install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.0MB/s 
     |████████████████████████████████| 460kB 45.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
! pip install gensim

In [4]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pB98oL46qBz_Y33qsXfQ94YjVe_YRWfU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pB98oL46qBz_Y33qsXfQ94YjVe_YRWfU" -O ratings_train.txt && rm -rf /tmp/cookies.txt
#https://drive.google.com/file/d/1pB98oL46qBz_Y33qsXfQ94YjVe_YRWfU/view?usp=sharing

--2021-05-17 10:22:26--  https://docs.google.com/uc?export=download&confirm=&id=1pB98oL46qBz_Y33qsXfQ94YjVe_YRWfU
Resolving docs.google.com (docs.google.com)... 74.125.129.102, 74.125.129.138, 74.125.129.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.129.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-0o-docs.googleusercontent.com/docs/securesc/oa6mus7qt5q2mr2ibk6e1jpkr2gkabr4/6vp6av0jetr866sfqtop4cqeicqbijdj/1621246875000/04591858293269658552/06706055881603712232Z/1pB98oL46qBz_Y33qsXfQ94YjVe_YRWfU?e=download [following]
--2021-05-17 10:22:27--  https://doc-10-0o-docs.googleusercontent.com/docs/securesc/oa6mus7qt5q2mr2ibk6e1jpkr2gkabr4/6vp6av0jetr866sfqtop4cqeicqbijdj/1621246875000/04591858293269658552/06706055881603712232Z/1pB98oL46qBz_Y33qsXfQ94YjVe_YRWfU?e=download
Resolving doc-10-0o-docs.googleusercontent.com (doc-10-0o-docs.googleusercontent.com)... 74.125.70.132, 2607:f8b0:4001:c02::84
Connecting to d

In [5]:
 import csv
 from konlpy.tag import Okt
 from gensim.models import word2vec
 import time

 #네이버 영화 코퍼스를 읽는다.
 f = open('ratings_train.txt', 'r', encoding='utf-8')
 rdr = csv.reader(f, delimiter='\t')
 rdw = list(rdr)
 f.close()

 #트위터 형태소 분석기를 로드한다. Twiter가 KoNLPy v0.4.5 부터 Okt로 변경 되었다.
 twitter = Okt()
print("Okt done")

start = time.time()
print("형태소 분석 시작")

 #텍스트를 한줄씩 처리합니다.
 result = []
 for line in rdw:
     #형태소 분석하기, 단어 기본형 사용
     malist = twitter.pos( line[1], norm=True, stem=True)
     r = []
     for word in malist:
         #Josa”, “Eomi”, “'Punctuation” 는 제외하고 처리
         if not word[1] in ["Josa","Eomi","Punctuation"]:
             r.append(word[0])
     #형태소 사이에 공백 " "  을 넣습니다. 그리고 양쪽 공백을 지웁니다.
     rl = (" ".join(r)).strip()
     result.append(rl)
     #print(rl)
end = time.time()         
print("형태소 분석 완료")
print(end - start, ' 초 결렸습니다')

 #형태소들을 별도의 파일로 저장 합니다.
 with open("NaverMovie.nlp",'w', encoding='utf-8') as fp:
     fp.write("\n".join(result))

 #Word2Vec 모델 만들기
 wData = word2vec.LineSentence("NaverMovie.nlp")

# 트레이닝 시간 측정 
start = time.time()
print("트레이닝 시작")
 wModel =word2vec.Word2Vec(wData, size=200, window=10, hs=1, min_count=2, sg=1)
 wModel.save("NaverMovie.model")
 print("Word2Vec 모델링 완료")
end = time.time()
print(end - start, ' 초 결렸습니다') 

Okt done
형태소 분석 시작
형태소 분석 완료
667.3108673095703  초 결렸습니다
트레이닝 시작
Word2Vec 모델링 완료
146.9395797252655  초 결렸습니다


In [6]:
! ls -l

total 123728
-rw-r--r-- 1 root root 100529895 May 17 10:36 NaverMovie.model
-rw-r--r-- 1 root root  11528154 May 17 10:33 NaverMovie.nlp
-rw-r--r-- 1 root root  14628807 May 17 10:22 ratings_train.txt
drwxr-xr-x 1 root root      4096 May  6 13:44 sample_data


In [7]:
model = word2vec.Word2Vec.load("NaverMovie.model")
print(model.wv.most_similar(positive=["최고"]))

[('최고다', 0.7046334147453308), ('꼽을', 0.6997150778770447), ('꼽는', 0.683068037033081), ('으뜸', 0.6783115863800049), ('하이스쿨', 0.6577041745185852), ('단연', 0.6565439701080322), ('정치드라마', 0.6561928987503052), ('손꼽다', 0.6402984261512756), ('꼽겠다', 0.6386947631835938), ('손에꼽는', 0.6382896304130554)]
